In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
import random
import time
import datetime
import sys

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



In [ ]:
# GPU가 맞게 설정됐는지 확인
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
  print(torch.cuda.get_device_name(i))

1
Tesla K80


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv
import pandas as pd

chatbot_data=pd.read_csv('/content/eda_data_1m.csv', encoding='utf-8')
chatbot_data.shape

(49595, 2)

In [ ]:
chatbot_data.sample(n=5)

,q,label
18765,광복절이후 풀린다고 한건 헛것을 본겐가?,0
17708,연아는 배우해도 되겠네..특별한얼굴이네,0
32429,이걸보는데 주책없이 눈물이 나네요~,0
19106,요새 왜이케 남자연예인 성폭행사건이 많이 일어나는지..,0
7064,기분나쁘게 듣지,1


In [ ]:
#이를 랜덤으로 섞어주고 train, test 나누기

chatbot_data_shuffled=chatbot_data.sample(frac=1).reset_index(drop=True)

train=chatbot_data_shuffled[:34300]
test=chatbot_data_shuffled[34300:]

In [ ]:
len(test)

15295

In [ ]:
# train set 전처리 : 문장의 앞에는 [CLS], 뒤에는 [SEP]를 붙임

sentences=["[CLS]"+ str(s)+"[SEP]" for s in train.q]

sentences[:5]

['[CLS]대상 수상 축하해요[SEP]',
 '[CLS]후회한다 아빠 잔소리 안 듣고 그러는 거 아빠 너 죽고[SEP]',
 '[CLS]40대이상 댓글 다 어디감?ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ[SEP]',
 '[CLS]우리때는 토하고 못 마셔도 술 와서 또 먹었어[SEP]',
 '[CLS]처음글써보는데 이런글을쓰게될줄은몰랏네;;[SEP]']

In [ ]:
labels=train['label'].values #train, test셋 분류 시를 위해 지정

In [ ]:
len(labels)

34300

## Subword Tokenizer : WordPiece

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
result=tokenizer.tokenize('안녕하세요')
print(result)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요']


단어집합이 비어있는 상태여서 단어가 한번 더 쪼개져서 ##이 붙음, 이는 어떠한 단어의 서브워드임을 뜻하고 이렇게 해서 원래 단어로 복원을 할 수 있게 된다

In [ ]:
#전체 데이터의 tokenize
tokenized_texts=[tokenizer.tokenize(s) for s in sentences]

print(sentences[0])
print(tokenized_texts[0])

[CLS]대상 수상 축하해요[SEP]
['[CLS]', '대', '##상', '수상', '축', '##하', '##해', '##요', '[SEP]']


In [ ]:
#최대 시퀀스 길이 설정 후 정수 인코딩과 제로패딩
# 정수 인코딩 : 각 단어에 고유한 정수를 부여
# 제로패딩 : 데이터에 0을 채워 데이터의 크기를 조정(길이가 동일해야 하나의 행렬로 취급할 수 있음)

max_len=128 # 최대 시퀀스 설정
input_ids=[tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts] # 정수 인코딩

input_ids=pad_sequences(input_ids, maxlen=max_len, dtype='long', truncating='post', padding='post') # 최대 시퀀스만큼 제로패딩

input_ids[0]

array([   101,   9069,  14871, 103938,   9766,  35506,  14523,  48549,
          102,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

## Attention Mask
0값을 가지는 패딩토큰에 대해 불필요한 어텐션 연산을 수행하지 않도록 단어와 패딩토큰을 구분함(패딩된 값 :0 , 패딩x:1)

In [ ]:
attention_masks=[]

for seq in input_ids:
  seq_mask=[float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [ ]:
attention_masks

[[1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0

In [ ]:
#train set > train, validation set

train_inputs, validation_inputs, train_labels, validation_labels=train_test_split(input_ids, labels, random_state=2000, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2000, test_size=0.1)

train_inputs=torch.tensor(train_inputs)
train_labels=torch.tensor(train_labels)
train_masks=torch.tensor(train_masks)
validation_inputs=torch.tensor(validation_inputs)
validation_labels=torch.tensor(validation_labels)
validation_masks=torch.tensor(validation_masks)

In [ ]:
# 배치사이즈를 설정하고 입력데이터, 어텐션마스크, 라벨을 하나의 데이터로 묶어 입력데이터 생성

batch_size=32

train_data=TensorDataset(train_inputs, train_masks, train_labels) #인덱싱과 슬라이스를 위한 방법 제공
train_sampler=RandomSampler(train_data)
train_dataloader=DataLoader(train_data, sampler=train_sampler, batch_size=32) # 배치사이즈에 따라 루프가 용이

validation_data=TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader=DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [ ]:
# train data 정제
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(s) + " [SEP]" for s in test.q]

# 라벨 데이터
labels = test['label'].values

# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 모델 생성

In [ ]:
# GPU설정을 위한 디바이스 설정

if torch.cuda.is_available():
  device=torch.device('cuda')
  print('There are %d GPU available.' % torch.cuda.device_count())

There are 1 GPU available.


In [ ]:
# pretrain된 bert모델 불러오기

model=BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 하이퍼 파라미터 설정

#optimizer
optimizer=AdamW(model.parameters(),
               lr=2e-5, #learnig rate
                eps=1e-8)

# epochs
epochs=4

# 총 훈련 스텝
total_steps=len(train_dataloader)*epochs

#scheduler
scheduler=get_linear_schedule_with_warmup(optimizer,
                                          num_warmup_steps=0,
                                          num_training_steps=total_steps)

## 모델 학습

In [ ]:
# 정확도 계산 함수

def flat_accuracy(preds, labels):
  pred_flat=np.argmax(preds, axis=1).flatten()
  labels_flat=labels.flatten()

  return np.sum(pred_flat == labels_flat)/len(labels_flat)

# 시간계산 함수

def format_time(elapsed):

  #반올림
  elapsed_rounded=int(round((elapsed)))

  #h:m:s 형태
  return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of    965.    Elapsed: 0:10:16.

  Average training loss: 0.16
  Training epcoh took: 0:19:48

Running Validation...
  Accuracy: 0.98
  Validation took: 0:00:46

======== Epoch 2 / 4 ========
Training...
  Batch   500  of    965.    Elapsed: 0:10:14.

  Average training loss: 0.07
  Training epcoh took: 0:19:45

Running Validation...
  Accuracy: 0.98
  Validation took: 0:00:46

======== Epoch 3 / 4 ========
Training...
  Batch   500  of    965.    Elapsed: 0:10:15.

  Average training loss: 0.04
  Training epcoh took: 0:19:54

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:47

======== Epoch 4 / 4 ========
Training...
  Batch   500  of    965.    Elapsed: 0:10:20.

  Average training loss: 0.02
  Training epcoh took: 0:19:58

Running Validation...
  Accuracy: 0.99
  Validation took: 0:00:47

Training complete!


In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    478.    Elapsed: 0:00:44.
  Batch   200  of    478.    Elapsed: 0:01:28.
  Batch   300  of    478.    Elapsed: 0:02:12.
  Batch   400  of    478.    Elapsed: 0:02:56.

Accuracy: 0.98
Test took: 0:03:31


In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks


In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    
    return logits

In [ ]:
logits = test_sentences(['기분 나쁘게 듣지 말고'])

print(logits)
if np.argmax(logits) == 1 :
  print("꼰대 관련 발언")
elif np.argmax(logits) == 0 :
  print("일상 발언")



[[-2.7671843  3.248907 ]]
꼰대 관련 발언


In [ ]:
logits = test_sentences(['여자친구한테 선물 뭘로 줄까?'])

print(logits)
if np.argmax(logits) == 1 :
    print("꼰대 관련 발언")
elif np.argmax(logits) == 0 :
    print("일상 발언")

[[ 4.333024 -4.767067]]
일상 발언
